In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# 1. 读取图像并转为灰度图
img = cv2.imread('input_cases/original.png', 0) # 0表示以灰度模式读取

# 2. 执行傅里叶变换
# 首先，为什么要用np.fft.fftshift？因为变换后的低频成分在四角，shift后能将其移到中心，便于观察。
dft = np.fft.fft2(img)
dft_shift = np.fft.fftshift(dft)

# 3. 计算幅度谱 (为了显示，我们取对数，因为动态范围太大)
magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1) # +1 防止log(0)

# 4. 可视化
plt.subplot(121), plt.imshow(img, cmap='gray')
plt.title('Input Image (Spatial)'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(magnitude_spectrum, cmap='gray')
plt.title('Magnitude Spectrum (Frequency)'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# 1. 读取图像并转为灰度图
img = cv2.imread('input_cases/recon.png', 0) # 0表示以灰度模式读取

# 2. 执行傅里叶变换
# 首先，为什么要用np.fft.fftshift？因为变换后的低频成分在四角，shift后能将其移到中心，便于观察。
dft = np.fft.fft2(img)
dft_shift = np.fft.fftshift(dft)

# 3. 计算幅度谱 (为了显示，我们取对数，因为动态范围太大)
magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1) # +1 防止log(0)

# 4. 可视化
plt.subplot(121), plt.imshow(img, cmap='gray')
plt.title('Input Image (Spatial)'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(magnitude_spectrum, cmap='gray')
plt.title('Magnitude Spectrum (Frequency)'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# 创建一个掩模，只保留中心区域（低频）
rows, cols = img.shape
crow, ccol = rows//2, cols//2
mask = np.zeros((rows, cols), np.uint8)
r = 50 # 滤波半径
mask[crow-r:crow+r, ccol-r:ccol+r] = 1

# 在频域应用掩模 (点乘)
fshift = dft_shift * mask

# 逆变换回空间域
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

# 可视化
plt.imshow(img_back, cmap='gray')
plt.title('Image after Low-pass Filtering')
plt.show()

In [ ]:
# 创建一个掩模，去除中心区域（低频）
mask = np.ones((rows, cols), np.uint8)
r = 50
mask[crow-r:crow+r, ccol-r:ccol+r] = 0
# 在频域应用掩模 (点乘)
fshift = dft_shift * mask

# 逆变换回空间域
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

# 可视化
plt.imshow(img_back, cmap='gray')
plt.title('Image after high-pass Filtering')
plt.show()
# ... 后续步骤同上

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from scipy.fftpack import dctn, idctn

def compute_spectral_differences(img1, img2):
    """
    计算两幅图像在频域上的多种差异指标
    """
    # 转换为灰度图
    if len(img1.shape) > 2:
        img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    if len(img2.shape) > 2:
        img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # 确保图像大小相同
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # 计算傅里叶变换和幅度谱
    dft1 = np.fft.fft2(img1.astype(float))
    dft2 = np.fft.fft2(img2.astype(float))
    
    magnitude1 = np.abs(np.fft.fftshift(dft1))
    magnitude2 = np.abs(np.fft.fftshift(dft2))
    
    # 为了避免log(0)，加一个小的epsilon
    eps = 1e-10
    log_magnitude1 = np.log(magnitude1 + eps)
    log_magnitude2 = np.log(magnitude2 + eps)
    
    # 计算各种差异指标
    differences = {}
    
    # 1. 整体频谱差异 (MSE in frequency domain)
    differences['spectral_mse'] = np.mean((magnitude1 - magnitude2) ** 2)
    differences['log_spectral_mse'] = np.mean((log_magnitude1 - log_magnitude2) ** 2)
    
    # 2. 高频能量损失比 (High-frequency Energy Loss Ratio)
    rows, cols = img1.shape
    crow, ccol = rows // 2, cols // 2
    
    # 创建高频掩模 (中心区域之外)
    mask_high = np.ones((rows, cols), bool)
    r_h, r_w = int(rows * 0.1), int(cols * 0.1)  # 中心10%区域为低频
    mask_high[crow-r_h:crow+r_h, ccol-r_w:ccol+r_w] = False
    
    # 创建低频掩模
    mask_low = ~mask_high
    
    # 计算各频带能量
    high_energy1 = np.sum(magnitude1[mask_high])
    high_energy2 = np.sum(magnitude2[mask_high])
    low_energy1 = np.sum(magnitude1[mask_low])
    low_energy2 = np.sum(magnitude2[mask_low])
    
    differences['high_freq_loss_ratio'] = 1 - (high_energy2 / high_energy1)
    differences['low_freq_loss_ratio'] = 1 - (low_energy2 / low_energy1)
    
    # 3. 频谱相关性 (Spectral Correlation)
    flat1 = magnitude1.flatten()
    flat2 = magnitude2.flatten()
    differences['spectral_correlation'] = np.corrcoef(flat1, flat2)[0, 1]
    
    return differences, magnitude1, magnitude2

# 使用示例
# 假设你有原图和VAE重建图
original_img = cv2.imread('input_cases/original.png', 0) # 灰度模式
reconstructed_img = cv2.imread('input_cases/recon.png', 0)

# resize reconstructed_img to match the size of original_img
reconstructed_img = cv2.resize(reconstructed_img, (original_img.shape[1], original_img.shape[0]))

# 计算频谱差异
diff_metrics, mag1, mag2 = compute_spectral_differences(original_img, reconstructed_img)

print("=== 频谱差异定量分析 ===")
for metric, value in diff_metrics.items():
    print(f"{metric}: {value:.6f}")

# 可视化对比
plt.figure(figsize=(15, 5))

plt.subplot(131)
plt.imshow(original_img, cmap='gray')
plt.title('Original Image')

plt.subplot(132)
plt.imshow(reconstructed_img, cmap='gray')
plt.title('Reconstructed Image')

plt.subplot(133)
# 显示频谱差异图
diff_spectrum = np.abs(mag1 - mag2)
plt.imshow(np.log(diff_spectrum + 1e-10), cmap='hot')
plt.title('Spectral Difference Map')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
def analyze_frequency_bands(img1, img2, band_ratios=[0.1, 0.2, 0.3, 0.4, 0.5]):
    """
    分析不同频带的能量分布
    """
    # 计算幅度谱
    dft1 = np.fft.fft2(img1.astype(float))
    magnitude1 = np.abs(np.fft.fftshift(dft1))
    
    dft2 = np.fft.fft2(img2.astype(float))
    magnitude2 = np.abs(np.fft.fftshift(dft2))
    
    rows, cols = img1.shape
    crow, ccol = rows // 2, cols // 2
    
    band_analysis = {'bands': [], 'energy_ratio1': [], 'energy_ratio2': []}
    
    total_energy1 = np.sum(magnitude1)
    total_energy2 = np.sum(magnitude2)
    
    prev_radius = 0
    for ratio in band_ratios:
        # 计算当前频带半径
        radius = int(min(rows, cols) * ratio / 2)
        
        # 创建环形掩模
        y, x = np.ogrid[:rows, :cols]
        mask = ((x - ccol)**2 + (y - crow)**2 >= prev_radius**2) & \
               ((x - ccol)**2 + (y - crow)**2 < radius**2)
        
        # 计算频带能量
        band_energy1 = np.sum(magnitude1[mask])
        band_energy2 = np.sum(magnitude2[mask])
        
        band_analysis['bands'].append(f'{prev_radius}-{radius}')
        band_analysis['energy_ratio1'].append(band_energy1 / total_energy1)
        band_analysis['energy_ratio2'].append(band_energy2 / total_energy2)
        
        prev_radius = radius
    
    return band_analysis

# 使用频带分析
band_analysis = analyze_frequency_bands(original_img, reconstructed_img)

# 绘制频带能量分布图
plt.figure(figsize=(10, 6))
x_pos = np.arange(len(band_analysis['bands']))
width = 0.35

plt.bar(x_pos - width/2, band_analysis['energy_ratio1'], width, 
        label='Original', alpha=0.7)
plt.bar(x_pos + width/2, band_analysis['energy_ratio2'], width, 
        label='Reconstructed', alpha=0.7)

plt.xlabel('Frequency Bands (pixels from center)')
plt.ylabel('Energy Ratio')
plt.title('Energy Distribution Across Frequency Bands')
plt.xticks(x_pos, band_analysis['bands'], rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def dct_analysis(img1, img2, block_size=8):
    """
    在DCT域分析图像差异（模拟JPEG压缩分析）
    """
    # 计算全局DCT
    dct1 = dctn(img1.astype(float), norm='ortho')
    dct2 = dctn(img2.astype(float), norm='ortho')
    
    # 分析不同频率系数的保留情况
    coefficients_preserved = {}
    
    # 按之字形顺序分析（模拟JPEG量化）
    for freq_level in range(1, min(block_size, block_size) + 1):
        # 创建频率掩模（选择特定频率区域）
        mask = np.zeros_like(img1, bool)
        for i in range(freq_level):
            for j in range(freq_level):
                if i + j < freq_level:  # 之字形模式
                    mask[i::block_size, j::block_size] = True
        
        # 计算该频率级别的能量保留率
        energy1 = np.sum(np.abs(dct1[mask]))
        energy2 = np.sum(np.abs(dct2[mask]))
        
        coefficients_preserved[f'freq_{freq_level}'] = {
            'energy_ratio': energy2 / energy1 if energy1 > 0 else 0,
            'original_energy': energy1,
            'reconstructed_energy': energy2
        }
    
    return coefficients_preserved, dct1, dct2

# DCT分析
dct_results, dct_orig, dct_recon = dct_analysis(original_img, reconstructed_img)

print("=== DCT域分析结果 ===")
for freq, data in dct_results.items():
    print(f"{freq}: Energy Preservation = {data['energy_ratio']:.4f}")

In [ ]:
from pca_extractor import build_diff_n_components_image

build_diff_n_components_image(model_path="sd-legacy/stable-diffusion-v1-5", 
                              n_components=4, 
                              n_channels=4,
                              prefix="sd_full_channel")

In [ ]:
from spec_analysis import FrequencyDomainAnalyzer
import os
# Replace with your actual image paths
image_paths = [
    f'vis/image_{i}.png' for i in range(0, 16)  # Adjust based on your actual filenames
]

# Or use specific paths like:
# image_paths = ['path/to/your/image1.jpg', 'path/to/your/image2.jpg', ...]

# Image names (optional)
image_names = [f'vis/image_{i}.png' for i in range(0, 16)]

# Create analyzer and perform analysis
analyzer = FrequencyDomainAnalyzer(image_size=(256, 256))

# Check if images exist
existing_paths = []
existing_names = []
for path, name in zip(image_paths, image_names):
    if os.path.exists(path):
        existing_paths.append(path)
        existing_names.append(name)
    else:
        print(f"File not found: {path}")

analyzer.analyze_images(existing_paths, existing_names)

strategies = ['sequence', 'spectral_centroid', 'entropy', 'energy']

# for strategy in strategies:
#    print(f"\n{'='*50}")
#    print(f"PCA Visualization with {strategy} coloring")
#    print(f"{'='*50}")
#    analyzer.enhanced_pca_visualization(color_strategy=strategy)



# Generate all visualizations
analyzer.visualize_spectral_comparison()

# Generate quantitative report
analyzer.generate_quantitative_report()

In [ ]:
from spec_analysis import advanced_sequence_analysis, domain_specific_metrics

advanced_sequence_analysis(analyzer)
domain_specific_metrics(analyzer)

In [ ]:
from spec_analysis import FrequencyDomainAnalyzer
import os
# Replace with your actual image paths

# Create analyzer and perform analysis
analyzer = FrequencyDomainAnalyzer(image_size=(256, 256))

image_paths = [
    f'input_cases/original.png', 'input_cases/recon.png'
] + [
    f'vis/image_{i}.png' for i in range(0, 4)  # Adjust based on your actual filenames
]
image_names = [
    f'input_cases/original.png', 'input_cases/recon.png'
]+[
    f'vis/image_{i}.png' for i in range(0, 4)  # Adjust based on your actual filenames
]
# Check if images exist
existing_paths = []
existing_names = []
for path, name in zip(image_paths, image_names):
    if os.path.exists(path):
        existing_paths.append(path)
        existing_names.append(name)
    else:
        print(f"File not found: {path}")
analyzer.analyze_images(existing_paths, existing_names)

# Generate all visualizations
analyzer.visualize_spectral_comparison()

# Generate quantitative report
analyzer.generate_quantitative_report()